<a href="https://colab.research.google.com/github/middlefitting/ML_Study/blob/main/everyofml/section3/data_preprocessing_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing Tools

## Importing the libraries

In [154]:
"""
numpy: 배열을 가지고 작업
pandas: 데이터 세트 뿐만 아니라 특정 행렬 및 종속 벡터 생성
matplotlib: 차트를 보여준다
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Importing the dataset

In [155]:
# 데이터 셋 불러오기
dataset = pd.read_csv('Data.csv')

# 피쳐: 독립 변수, 종속 변수를 예측하는데 필요한 변수
# 추출하고자 하는 행, 열의 인덱스를 받는다
X = dataset.iloc[:, :-1].values

# 종속 변수: 예측하고자 하는 것
y = dataset.iloc[:, -1].values

In [156]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [157]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [158]:
print(y)

['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


## Taking care of missing data

결측치 처리방법
- 무시하고 삭제하기: 데이터 셋이 클 때에는 상관없다.
- 평균값으로 대치


In [159]:
missing_data = dataset.isnull().sum()
print(missing_data)

Country      0
Age          1
Salary       1
Purchased    0
dtype: int64


In [160]:
# 사이킷런 가져오기
from sklearn.impute import SimpleImputer

# 인스턴스 생성
# 대치할 값, 대치 전략
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# fit: 결측값을 찾아 평균을 계산, 대치하는 것은 아니다
imputer.fit(X[:, 1:3])

# transform: 결측값을 대치
X[:, 1:3] = imputer.transform(X[:, 1:3])

In [161]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Encoding categorical data

In [162]:
"""
원핫 인코딩
- 각 카테고리의 바이너리 벡터를 생성한다
- 범주형 데이터를 처리하는데 있어 유용하다

"""

'\n원핫 인코딩\n- 각 카테고리의 바이너리 벡터를 생성한다\n- 범주형 데이터를 처리하는데 있어 유용하다\n\n'

### Encoding the Independent Variable

In [163]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# 인자: 어떻게 변환하고 어떤 열 인덱스를 변환할지 명시하는 트랜스포머, 변환을 하지 않을 열 명시
# ramainder='passthrough' 의미는 원핫인코딩 하지 않은 열들도 반환하라는 것이다
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
# 넘파이 배열을 반환하는 것이 아니라서 나중에 fit이라는 함수로 전달하려면 넘파이 배열로 변환해야 한다
X = np.array(ct.fit_transform(X))
print(X)


[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


### Encoding the Dependent Variable

In [164]:
from sklearn.preprocessing import LabelEncoder

# 이진 데이터의 경우 활용할 수 있는 옵션
le = LabelEncoder()
y = le.fit_transform(y)

print(y)

[0 1 0 0 1 1 0 1 0 1]


## Splitting the dataset into the Training set and Test set

In [165]:
"""
데이터 분리 이후 피쳐 스케일링을 적용해야 한다.

데이터 세트를 나눈다는 것은 관측치 성능 평가가 테스트 세트이다
피쳐 스케일링은 모든 특성을 동일한 스케일로 변환할 뿐이다. 값이 작아서 머신 머닝 모델이 무시할 수 있는 내용을 처리해주는 것이다..

단순히 테스트 세트가 머신러닝 모델을 평가하기 위한 녀석이어야 하고 훈련 세트와 인과관계가 없어야 한다.

그런데 피쳐 스케일링은 특성의 평균과 표준편차를 사용하는 기법이다.
따라서 테스트 세트를 나누기 전에 스케일링을 적용하면 표준편차를 사용하게 되어버리는 것이다
"""

'\n데이터 분리 이후 피쳐 스케일링을 적용해야 한다.\n\n데이터 세트를 나눈다는 것은 관측치 성능 평가가 테스트 세트이다\n피쳐 스케일링은 모든 특성을 동일한 스케일로 변환할 뿐이다. 값이 작아서 머신 머닝 모델이 무시할 수 있는 내용을 처리해주는 것이다..\n\n단순히 테스트 세트가 머신러닝 모델을 평가하기 위한 녀석이어야 하고 훈련 세트와 인과관계가 없어야 한다.\n\n그런데 피쳐 스케일링은 특성의 평균과 표준편차를 사용하는 기법이다.\n따라서 테스트 세트를 나누기 전에 스케일링을 적용하면 표준편차를 사용하게 되어버리는 것이다\n'

In [166]:
from sklearn.model_selection import train_test_split

# 동일한 결과를 보기 위해 시드를 고정
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)


In [167]:
print(X_train)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]


In [168]:
print(X_test)

[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [169]:
print(y_train)

[0 1 0 0 1 1 0 1]


In [170]:
print(y_test)

[0 1]


## Feature Scaling

In [171]:
"""
이유: 특정 피쳐가 무시될 수 있기 때문이다. 값이 작은 피쳐는 피쳐 스케일링을 하지 않으면 모델에서 무시될 가능성이 크다

방법
- 표준화: 평균을 빼고 분산의 제곱근으로 나눈다 -> -3 ~ 3 으로 조정된다
- 정규화: 최솟값을 빼고 최댓값에서 최솟값을 뺀 값으로 나눈다 -> 0 ~ 1로 조정된다

무엇을 해야 하는가
- 정규화: 대부분의 피쳐가 정규분포를 따른다면 정규화가 좋다
- 표준화: 대부분 좋다
- 결론: 표준화를 택하는 것이 권장된다. 항상 의미 있는 스케일링이 되기 때문이다.

과정
- 훈련 데이터의 평균과 표준편차를 구하고 훈련 및 테스트 셋에 적용한다.
- 테스트 셋의 평균 및 표준편차 값을 사용하지 않게 주의한다.
"""

'\n이유: 특정 피쳐가 무시될 수 있기 때문이다. 값이 작은 피쳐는 피쳐 스케일링을 하지 않으면 모델에서 무시될 가능성이 크다\n\n방법\n- 표준화: 평균을 빼고 분산의 제곱근으로 나눈다 -> -3 ~ 3 으로 조정된다\n- 정규화: 최솟값을 빼고 최댓값에서 최솟값을 뺀 값으로 나눈다 -> 0 ~ 1로 조정된다\n\n무엇을 해야 하는가\n- 정규화: 대부분의 피쳐가 정규분포를 따른다면 정규화가 좋다\n- 표준화: 대부분 좋다\n- 결론: 표준화를 택하는 것이 권장된다. 항상 의미 있는 스케일링이 되기 때문이다.\n\n과정\n- 훈련 데이터의 평균과 표준편차를 구하고 훈련 및 테스트 셋에 적용한다.\n- 테스트 셋의 평균 및 표준편차 값을 사용하지 않게 주의한다.\n'

In [172]:
from sklearn.preprocessing import StandardScaler

# 파라미터는 필요없다. 자동으로 적용된다.
sc = StandardScaler()

# 특성 행렬에도 적용해야 하는가 -> No. 표준화의 목적은 모든 특성의 값을 변환하는 것인데 가변수는 이미 1과 0으로 구상되므로 의미없다
# 오히려 성능이 떨어질 수 있다. 값이 달라지므로 의미가 사라진다
# 숫자형 값에만 사용한다.
# fit: 평균과 표준편차를 구한다, transform: 적용한다
X_train[:, 3:] = sc.fit_transform(X_train[:, 3:])
# 스케일러를 그대로 사용해야 하니까 다시 fit 하면 안된다
X_test[:, 3:] = sc.transform(X_test[:, 3:])

In [173]:
print(X_train)

[[0.0 0.0 1.0 -0.19159184384578545 -1.0781259408412425]
 [0.0 1.0 0.0 -0.014117293757057777 -0.07013167641635372]
 [1.0 0.0 0.0 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.0 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.0 -1.9018011447007988 -1.420463615551582]
 [1.0 0.0 0.0 1.1475343068237058 1.232653363453549]
 [0.0 1.0 0.0 1.4379472069688968 1.5749910381638885]
 [1.0 0.0 0.0 -0.7401495441200351 -0.5646194287757332]]


In [174]:
print(X_test)

[[0.0 1.0 0.0 -1.4661817944830124 -0.9069571034860727]
 [1.0 0.0 0.0 -0.44973664397484414 0.2056403393225306]]
